In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import requests
import urllib, json
from PIL import Image
import datetime

In [59]:
# Подключаем api_key из файла
f = open('API_KEY.txt', 'r')
api_key = f.read()
f.close()

In [60]:
# Составляем URL request
api_root = 'https://api.nasa.gov/'
api_product = 'neo/rest/v1/feed?'
start_date = '2020-07-07'
end_date = '2020-07-27'
url = f"{api_root}{api_product}start_date={start_date}&end_date={end_date}&api_key={api_key}"

In [61]:
'''
    Функция навигации по данным из api response
    возвращает ссылку на данные в другом временном отрезке

    step = 'next' -- следующий день от заданного start_date в URL request
    step = 'prev' -- предыдущий день от заданного start_date в URL request
    step = 'self' -- текущий день, равный start_date в URL request
'''

def get_url(url, step):
    url_level = urllib.request.urlopen(url)
    data_level = json.loads(url_level.read())['links']
    if step == 'next':
        return data_level[step]
    elif step == 'prev':
        return data_level[step]
    elif step == 'self':
        return data_level[step]

In [62]:
delta = datetime.timedelta(days=1)
day_current = datetime.date.fromisoformat(start_date)
date_str = datetime.date.strftime(a, '%Y-%m-%d')

In [63]:
# Делаем список URL-запросов из 100 дней
#for i in range(1, 101):
#    day_current += delta
#    url_current = f"{api_root}{api_product}start_date={day_current}&end_date={end_date}&api_key={api_key}"
#    print(url_current)


In [64]:
# Навигация по ссылкам
# Работаем с данными за один день 2020-07-07
url_level_0 = urllib.request.urlopen(url)
data_level_0 = pd.DataFrame.from_dict(json.loads(url_level_0.read()))
data_level_0

HTTPError: HTTP Error 400: Bad Request

In [ ]:
# Выбираем один элемент из списка за один день
data_day = data_level_0.near_earth_objects[3]
data_day_item = data_day[0]
data_day_item

In [ ]:
# Создаем словари с выбранными данными для DataFrame
dict_estimated_diameter = data_day_item['estimated_diameter']['kilometers']
dict_close_approach_data = data_day_item['close_approach_data']
dict_relative_velocity = dict_close_approach_data[0]['relative_velocity']
dict_miss_distance = dict_close_approach_data[0]['miss_distance']

In [ ]:
# Создаем DataFrame
# Объединяем DataFrame
df1 = pd.DataFrame(data_day_item, index=['1'])
df2 = pd.DataFrame(dict_estimated_diameter, index=['1'])
df3 = pd.DataFrame(dict_close_approach_data, index=['1'])
df4 = pd.DataFrame(dict_relative_velocity, index=['1'])
df5 = pd.DataFrame(dict_miss_distance, index=['1'])
df1.join(df2).join(df3).join(df4).join(df5)

In [ ]:
# Чистим DataFrame от нерелевантных данных, переименовываем
df_item = df1.join(df2).join(df3).join(df4).join(df5).drop(['links',
                                                  'nasa_jpl_url',
                                                  'neo_reference_id',
                                                  'estimated_diameter',
                                                  'close_approach_data',
                                                  'relative_velocity',
                                                  'miss_distance',
                                                  'miles_per_hour',
                                                  'astronomical',
                                                  'lunar',
                                                  'miles'
                                                  ], axis=1
).rename(columns={'kilometers': 'miss_distance_kilometers',
                  'estimated_diameter_min': 'estimated_diameter_km_min',
                  'estimated_diameter_max': 'estimated_diameter_km_max'})

df_item